# async, await

ref :
- http://postd.cc/python-generators-coroutines-native-coroutines-and-async-await/
- http://blanktar.jp/blog/2015/07/python-yield-from.html

## setting

In [4]:
%load_ext autoreload

In [5]:
%autoreload 2

## generater

In [2]:
def generate_nums():
    num = 0
    while True:
        yield num
        print("Num: ", num)
        num = num + 1
 
 
nums = generate_nums()
 
for x in nums:
    print(x)
    print("="*9)
 
    if x > 9:
        break

0
Num:  0
1
Num:  1
2
Num:  2
3
Num:  3
4
Num:  4
5
Num:  5
6
Num:  6
7
Num:  7
8
Num:  8
9
Num:  9
10


In [7]:
def coro():
    hello = yield "Hello"
    yield hello
 
c = coro()

In [8]:
print(next(c))

Hello


In [67]:
print(c.send("World"))

World


### ジェネレーターとメモリ消費量
ジェネレーターはメモリ消費量が少なくすむ。
リスト内包表記でもしかり

In [7]:
from memory_profiler import profile

In [6]:
def tmp():
    print("hoge")

In [7]:
%load_ext memory_profiler

In [20]:
iter_num = 10000000

In [21]:
%memit
len([x for x in range(iter_num) if x % 3 == 0])

peak memory: 44.48 MiB, increment: 0.00 MiB


3333334

In [22]:
%memit
sum((1 for x in range(iter_num) if x % 3 == 0))

peak memory: 47.77 MiB, increment: 0.00 MiB


3333334

## what is 'yield from'

In [41]:
def range_ten():
    #yield from range(10)
    num = 0
    while True:
        yield num
        if num == 10:
            break
        num += 1
        
def range_ten_from():
    yield from range(10)

In [36]:
x = range_ten()
for i in x:
    if i == 5:
        break
    print(i)

0
1
2
3
4


In [50]:
x =range_ten_from()
next(x)

0

In [54]:
next(x)

4

In [55]:
next(x)

5

In [57]:
def tea():
    yield from range(0,4)
    return "Hello"

In [58]:
def teb():
    x = yield from tea()
    print(x)


In [64]:
x = tea()

In [65]:
list(x)

[0, 1, 2, 3]

# asyncio (python3.4~)

In [69]:
import asyncio
import datetime
import random
 
 
@asyncio.coroutine
def display_date(num, loop):
    end_time = loop.time() + 10.0
    while True:
        print("Loop: {} Time: {}".format(num, datetime.datetime.now()))
        if (loop.time() + 1.0) >= end_time:
            break
        yield from asyncio.sleep(random.randint(0, 5))
 
 
loop = asyncio.get_event_loop()
 
asyncio.ensure_future(display_date(1, loop))
asyncio.ensure_future(display_date(2, loop))
 
loop.run_forever()

Loop: 1 Time: 2016-12-20 17:25:49.858244
Loop: 2 Time: 2016-12-20 17:25:49.858428
Loop: 1 Time: 2016-12-20 17:25:49.858551
Loop: 2 Time: 2016-12-20 17:25:49.858612
Loop: 1 Time: 2016-12-20 17:25:49.858651
Loop: 1 Time: 2016-12-20 17:25:51.859859
Loop: 2 Time: 2016-12-20 17:25:53.863919
Loop: 1 Time: 2016-12-20 17:25:53.864109
Loop: 2 Time: 2016-12-20 17:25:53.864237
Loop: 2 Time: 2016-12-20 17:25:56.869681
Loop: 1 Time: 2016-12-20 17:25:58.865731
Loop: 2 Time: 2016-12-20 17:26:01.875245


KeyboardInterrupt: 

In [113]:
print(next(display_date(2, loop)))

Loop: 2 Time: 2016-12-20 17:31:00.023066
<Future pending>


In [117]:
num =0
for i in asyncio.sleep(0.1):
    if num == 10:
        break
    print(num, ": ", i)

0 :  <Future pending>


AssertionError: yield from wasn't used with future

In [118]:
asyncio.sleep(0.1)

<generator object sleep at 0x1046c1fc0>